In [1]:
import os
import sys
BASE_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, os.path.join(BASE_DIR))
from offline import SparkSessionBase

In [2]:
class UpdateUserProfile(SparkSessionBase):
    
    SPARK_APP_NAME = "updateUser"
    ENABLE_HIVE_SUPPORT = True
    
    def __init__(self):
        self.spark = self._create_spark_session()
        
uup = UpdateUserProfile()

In [3]:
uup.spark.sql("use fytang")
user_article_basic = uup.spark.sql("select * from user_article_basic")
article_profile = uup.spark.sql("select * from article_top")

In [4]:
user_topic_res = user_article_basic.join(article_profile, on=['article_id'])

In [5]:
user_topic_res.show()

+----------+-------------------+-------------------+----------+------+-------+---------+--------+---------+--------+
|article_id|            user_id|        action_time|channel_id|shared|clicked|collected|exposure|read_time|   topic|
+----------+-------------------+-------------------+----------+------+-------+---------+--------+---------+--------+
|     13401|1106396183141548032|2019-03-28 10:58:20|         0| false|  false|    false|    true|         |      补码|
|     13401|1106396183141548032|2019-03-28 10:58:20|         0| false|  false|    false|    true|         |     字符串|
|     13401|1106396183141548032|2019-03-28 10:58:20|         0| false|  false|    false|    true|         |      李白|
|     13401|1106396183141548032|2019-03-28 10:58:20|         0| false|  false|    false|    true|         |    type|
|     13401|1106396183141548032|2019-03-28 10:58:20|         0| false|  false|    false|    true|         |      元素|
|     13401|1106396183141548032|2019-03-28 10:58:20|         0| 

In [15]:
def compute_weights(partitions):
    
    weightsOfaction = {
        "read_min" : 1,
        "read_middle" : 2,
        "collect" : 2,
        "share" : 3,
        "click" : 5
    }
    
    import happybase
    from datetime import datetime
    import numpy as np
    
#     pool = happybase.ConnectionPool(size=10, host='127.0.0.1', port=9090)
    pool = happybase.ConnectionPool(size=10, host='localhost',port=9090)

    for row in partitions:
        t = datetime.now() - datetime.strptime(row.action_time, '%Y-%m-%d %H:%M:%S')
        time_expr = 1 / (np.log(t.days + 1) + 1)
        
        if row.read_time == "":
            r_t = 0
        else:
            r_t = int(row.read_time)
            
        read_weights = weightsOfaction['read_middle'] if r_t > 1000 else weightsOfaction['read_min']
        topic_weights = time_expr * (row.shared * weightsOfaction['share'] + row.clicked * weightsOfaction['click'] +
                                     row.collected * weightsOfaction['collect'] + read_weights
                                    )
        with pool.connection as conn:
            table = conn.table('user_profile')
            table.put("user:{}".format(row.user_id).encode(),{'partial:{}:{}'.format(row.channel_id,topic).encode(): str(topic_weights).encode()})
            conn.close()
                               
user_topic_res.foreachPartition(compute_weights)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 32.0 failed 4 times, most recent failure: Lost task 5.3 in stage 32.0 (TID 200, 192.168.199.174, executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark-2.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/usr/local/spark-2.1.1-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/local/spark-2.1.1-bin-hadoop2.7/python/pyspark/rdd.py", line 2408, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark-2.1.1-bin-hadoop2.7/python/pyspark/rdd.py", line 2408, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/local/spark-2.1.1-